In [1]:
import boto3, re
from sagemaker import get_execution_role

In [2]:
role = get_execution_role()

In [3]:
role

'arn:aws:iam::174390337627:role/service-role/AmazonSageMaker-ExecutionRole-20190115T134792'

In [4]:
import keras
from keras.models import model_from_json

Using TensorFlow backend.


In [5]:
json_file = open('selfie_model/'+'selfie_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
loaded_model.load_weights('selfie_model/selfie_model-weights.h5')

In [7]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [8]:
%%time
model_version = '039521'

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [9]:
export_dir = 'export/Servo/' + model_version

In [10]:
builder = builder.SavedModelBuilder(export_dir)

In [11]:
signature = predict_signature_def(inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [12]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

In [13]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [14]:
import sagemaker
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='Robert/model/image')

In [15]:
!cat train.py

In [16]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/Robert/model/image/model.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')
#sagemaker_model.fit()

In [17]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-2019-03-25-02-11-36-553
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2019-03-25-02-11-36-553


---------------------------------------------------------------------------!CPU times: user 494 ms, sys: 34.2 ms, total: 528 ms
Wall time: 6min 22s


In [18]:
endpoint_name = "sagemaker-tensorflow-2019-03-25-02-11-36-553"
import sagemaker
from sagemaker.tensorflow.model import TensorFlowPredictor
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

In [19]:
!pip install opencv-python
from PIL import Image
#import tensorflow as tf
import numpy as np
import cv2
#from tensorflow.core.framework import tensor_pb2

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
IMG_SIZE = 256
im = cv2.imread("Positive-(11).jpg")
im_r = cv2.resize(im, (256,256))
im_r = cv2.cvtColor(im_r, cv2.COLOR_BGR2GRAY)
im_t = im_r.reshape(1,256,256,1).astype('float32')
type(im_t)

numpy.ndarray

In [21]:
pic = Image.open("negative-(620).jpg")
pic = pic.convert('L')
pic = pic.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
pix = np.array(pic.getdata()).reshape(1, 256, 256, 1)
pin = pix.tobytes()
type(pin)
pix

array([[[[21],
         [22],
         [22],
         ...,
         [14],
         [14],
         [15]],

        [[23],
         [22],
         [20],
         ...,
         [15],
         [15],
         [13]],

        [[23],
         [24],
         [23],
         ...,
         [13],
         [14],
         [10]],

        ...,

        [[17],
         [19],
         [19],
         ...,
         [13],
         [12],
         [10]],

        [[16],
         [12],
         [13],
         ...,
         [12],
         [13],
         [12]],

        [[10],
         [ 8],
         [10],
         ...,
         [12],
         [15],
         [12]]]])

In [22]:
import boto3
runtime = boto3.Session().client(service_name='sagemaker-runtime', region_name='ap-southeast-1') 
import json
import numpy as np
#data = np.random.randn(1, 256, 256, 3)
predictor.predict(pix)

{'outputs': {'score': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 2}]},
   'float_val': [5.789118506403443e-14, 1.0]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 21039524},
  'signature_name': 'serving_default'}}

In [23]:
img_predict=predictor.predict(im_t)

In [24]:
type(img_predict)

dict

In [25]:
import os

In [26]:
files = os.listdir("images")

In [27]:
for file in files:
    pic = Image.open("images/"+file)
    pic = pic.convert('L')
    pic = pic.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
    pix = np.array(pic.getdata()).reshape(1, 256, 256, 1)
    predict_image=predictor.predict(pix)
    print(file, predict_image)

not_selfie (1701).jpg {'outputs': {'score': {'dtype': 1, 'tensor_shape': {'dim': [{'size': 1}, {'size': 2}]}, 'float_val': [0.9999711513519287, 2.8834725526394323e-05]}}, 'model_spec': {'name': 'generic_model', 'version': {'value': 21039524}, 'signature_name': 'serving_default'}}
not_selfie (1702).jpg {'outputs': {'score': {'dtype': 1, 'tensor_shape': {'dim': [{'size': 1}, {'size': 2}]}, 'float_val': [0.999211311340332, 0.0007886163657531142]}}, 'model_spec': {'name': 'generic_model', 'version': {'value': 21039524}, 'signature_name': 'serving_default'}}
not_selfie (1703).jpg {'outputs': {'score': {'dtype': 1, 'tensor_shape': {'dim': [{'size': 1}, {'size': 2}]}, 'float_val': [0.9999921321868896, 7.880856173869688e-06]}}, 'model_spec': {'name': 'generic_model', 'version': {'value': 21039524}, 'signature_name': 'serving_default'}}
selfie_images (6).jpg {'outputs': {'score': {'dtype': 1, 'tensor_shape': {'dim': [{'size': 1}, {'size': 2}]}, 'float_val': [5.160071304999292e-05, 0.99994838237